In [ ]:
!pip install pandas folium geopandas


In [11]:
import pandas as pd
import folium
# from geopy.geocoders import Nominatim
# from geopy.extra.rate_limiter import RateLimiter

In [12]:
df = pd.read_csv('/content/pet-ownership-statistics-by-country-2024.csv')

df.head()

,country,PetOwnershipDogPopulation,PetOwnershipCatPopulation,PetOwnershipBirdPopulation,PetOwnershipAquariaPopulation,PetOwnershipSmallMammalPopulation,PetOwnershipTerrariaPopulation
0,India,10200000.0,NaN,NaN,NaN,NaN,NaN
1,China,27400000.0,53100000.0,NaN,NaN,NaN,NaN
2,United States,69929000.0,74059000.0,8300000.0,NaN,NaN,NaN
3,Brazil,NaN,NaN,191001000.0,NaN,NaN,NaN
4,Russia,NaN,23150000.0,3280000.0,3750000.0,6601000,1076000.0


In [13]:
# Convert relevant columns, forcing non-numeric values to NaN
columns_to_convert = [
    'PetOwnershipDogPopulation', 'PetOwnershipCatPopulation', 'PetOwnershipBirdPopulation',
    'PetOwnershipAquariaPopulation', 'PetOwnershipSmallMammalPopulation', 'PetOwnershipTerrariaPopulation'
]
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce').fillna(0)

In [14]:
# Add coordinates latitude and longitude for the countries
# note- use api as list grows
coordinates = {
    'India': [20.5937, 78.9629],
    'China': [35.8617, 104.1954],
    'United States': [37.0902, -95.7129],
    'Brazil': [-14.2350, -51.9253],
    'Russia': [61.5240, 105.3188],
    'Ethiopia': [9.145, 40.4897],
    'Japan': [36.2048, 138.2529],
    'Philippines': [12.8797, 121.7740],
    'Turkey': [38.9637, 35.2433],
    'Germany': [51.1657, 10.4515],
    'United Kingdom': [55.3781, -3.4360],
    'France': [46.6034, 1.8883],
    'South Africa': [-30.5595, 22.9375],
    'Italy': [41.8719, 12.5674],
    'Spain': [40.4637, -3.7492],
    'Argentina': [-38.4161, -63.6167],
    'Poland': [51.9194, 19.1451],
    'Ukraine': [48.3794, 31.1656],
    'Australia': [-25.2744, 133.7751],
    'Romania': [45.9432, 24.9668],
    'Netherlands': [52.1326, 5.2913],
    'Belgium': [50.8503, 4.3517],
    'Sweden': [60.1282, 18.6435],
    'Czech Republic': [49.8175, 15.4730],
    'Greece': [39.0742, 21.8243],
    'Portugal': [39.3999, -8.2245],
    'Hungary': [47.1625, 19.5033],
    'Austria': [47.5162, 14.5501],
    'Switzerland': [46.8182, 8.2275],
    'Bulgaria': [42.7339, 25.4858],
    'Denmark': [56.2639, 9.5018],
    'Slovakia': [48.6690, 19.6990],
    'Finland': [61.9241, 25.7482],
    'Norway': [60.4720, 8.4689],
    'Ireland': [53.1424, -7.6921],
    'Lithuania': [55.1694, 23.8813],
    'Slovenia': [46.1512, 14.9955],
    'Latvia': [56.8796, 24.6032],
    'Estonia': [58.5953, 25.0136]
}


In [15]:
# #   future notes idea
# geolocator
# geolocator = Nominatim(user_agent="geoapiExercises")
# geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# # coordinates for each country
# def get_coordinates(country):
#     try:
#         location = geolocator.geocode(country)
#         return location.latitude, location.longitude
#     except:
#         return None, None

# # Apply the function to fetch coordinates
# df['latitude'], df['longitude'] = zip(*df['country'].apply(get_coordinates))



In [16]:
# map centered around a global location
map_center = [20.0, 0.0]  # Rough center of the world map
m = folium.Map(location=map_center, zoom_start=3)


# columns for latitude and longitude
df['latitude'] = df['country'].map(lambda x: coordinates.get(x, [None, None])[0])
df['longitude'] = df['country'].map(lambda x: coordinates.get(x, [None, None])[1])

In [17]:
#pet ownership
for idx, row in df.dropna(subset=['latitude', 'longitude']).iterrows():
    popup_text = f"""
    Country: {row['country']}<br>
    Dogs: {row['PetOwnershipDogPopulation']}<br>
    Cats: {row['PetOwnershipCatPopulation']}<br>
    Birds: {row['PetOwnershipBirdPopulation']}<br>
    Aquaria: {row['PetOwnershipAquariaPopulation']}<br>
    Small Mammals: {row['PetOwnershipSmallMammalPopulation']}<br>
    Terraria: {row['PetOwnershipTerrariaPopulation']}
    """
    # Calculate total population, replacing NaN with 0
    total_population = sum([
        row['PetOwnershipDogPopulation'], row['PetOwnershipCatPopulation'],
        row['PetOwnershipBirdPopulation'], row['PetOwnershipAquariaPopulation'],
        row['PetOwnershipSmallMammalPopulation'], row['PetOwnershipTerrariaPopulation']
    ])

    # Ensure radius is positive
    radius = max((total_population ** 0.5) / 1000, 1)

    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=radius,  # Adjust radius for better visualization
        popup=popup_text,
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_to(m)


In [18]:
#save map to html file
m.save('global_pet_ownership_map.html')